In [128]:
import os
from abc import ABCMeta, abstractmethod
from collections import OrderedDict
import subprocess
import torch
import shutil
from PIL import Image
from pdb import set_trace as st
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter

In [83]:
def create_tasks(num_tasks=10, path="/root/yifei/CL_mod/src/dataset/tiny-imagenet-200/tiny-imagenet-200", seed=0):
    # This function creates the corresponding tasks dictionary which maps class_name: task_id
     
    # Get the file path that stores the class names
    file_path = os.path.join(root_path, "wnids.txt")
    
    # Get the class names as a list
    lines = [line.rstrip('\n') for line in open(file_path)]
    
    # Make sure the tasks are correctly assigned
    nb_classes_task = len(lines) // num_tasks
    print("Split "+str(len(lines))+" classes into "+ str(num_tasks)+" tasks with "+str(nb_classes_task)+" classes per task")
    assert len(lines) % num_tasks == 0, "total "+str(len(lines))+" classes must be divisible by nb classes per task"
    
    # Create a dictionary to return in this format: task_number -> class_name
    out_dict = {}
    for i in range(num_tasks):
        out_dict[i] = lines[i*num_tasks:(i+1)*num_tasks]
        
    return out_dict

In [85]:
create_tasks()

Split 200 classes into 10 tasks with 20 classes per task


{0: ['n02124075',
  'n04067472',
  'n04540053',
  'n04099969',
  'n07749582',
  'n01641577',
  'n02802426',
  'n09246464',
  'n07920052',
  'n03970156'],
 1: ['n03891332',
  'n02106662',
  'n03201208',
  'n02279972',
  'n02132136',
  'n04146614',
  'n07873807',
  'n02364673',
  'n04507155',
  'n03854065'],
 2: ['n03838899',
  'n03733131',
  'n01443537',
  'n07875152',
  'n03544143',
  'n09428293',
  'n03085013',
  'n02437312',
  'n07614500',
  'n03804744'],
 3: ['n04265275',
  'n02963159',
  'n02486410',
  'n01944390',
  'n09256479',
  'n02058221',
  'n04275548',
  'n02321529',
  'n02769748',
  'n02099712'],
 4: ['n07695742',
  'n02056570',
  'n02281406',
  'n01774750',
  'n02509815',
  'n03983396',
  'n07753592',
  'n04254777',
  'n02233338',
  'n04008634'],
 5: ['n02823428',
  'n02236044',
  'n03393912',
  'n07583066',
  'n04074963',
  'n01629819',
  'n09332890',
  'n02481823',
  'n03902125',
  'n03404251'],
 6: ['n09193705',
  'n03637318',
  'n04456115',
  'n02666196',
  'n03796401'

In [120]:
class tinyImageNet(torchvision.datasets.ImageFolder):
    # Class that inherits from imagefolder for dataloading purposes
    # Applies basic transforms and accepts a subset of classes for training/testing
    # Modifies the find_classes function from torch's Dataset class
    # Remember in defying paper, train is split 80:20 to train/val; val is for testing
    def __init__(self, root="/root/yifei/CL_mod/src/dataset/tiny-imagenet-200/tiny-imagenet-200/train", 
                 transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),]), 
                 target_transform=None, subset=None):
        # Subset stores a dictionary of class names to its label
        self.subset = subset
        super().__init__(root=root, transform=transform, target_transform=target_transform)
        self.all_classes = torchvision.datasets.folder.find_classes(root)
        
    def find_classes(self, path):
        if self.subset:
        # If using a subset of all classes, then only use those classes
            return self.subset.keys(), self.subset 
        else:
        # Else, use torch's generalized class function
            return torchvision.datasets.folder.find_classes(path)
    

In [121]:
def create_tasks(num_tasks=10, path="/root/yifei/CL_mod/src/dataset/tiny-imagenet-200/tiny-imagenet-200"):
    # Splitting into n classes creates a n-length array, where each item in the array consists of
    # a dictionary that maps class_labels to class_id (ie 0)
     
    # Get the file path that stores the class names
    file_path = os.path.join(root_path, "wnids.txt")
    
    # Get the class names as a list
    lines = [line.rstrip('\n') for line in open(file_path)]
    
    # Make sure the tasks are correctly assigned
    nb_classes_task = len(lines) // num_tasks
    print("Split "+str(len(lines))+" classes into "+ str(num_tasks)+" tasks with "+str(nb_classes_task)+" classes per task")
    assert len(lines) % num_tasks == 0, "total "+str(len(lines))+" classes must be divisible by nb classes per task"
    
    # Create a dictionary to return in this format: task_number -> class_name
    outputs = []
    current_id = 0
    for i in range(num_tasks):
        class_lbl = lines[i*num_tasks:(i+1)*num_tasks]  
        task_dict = {} 
        for j in range(i*num_tasks, (i+1)*num_tasks):
            task_dict[lines[j]] = current_id
            current_id += 1
        outputs.append(task_dict)
        
    return outputs

In [122]:
f = create_tasks()

Split 200 classes into 10 tasks with 20 classes per task


In [123]:
e = tinyImageNet(subset=f[1])
train_loader = torch.utils.data.DataLoader(e, batch_size=64, shuffle=True)

In [129]:
Counter(e.targets)

Counter({11: 500,
         14: 500,
         13: 500,
         17: 500,
         12: 500,
         19: 500,
         10: 500,
         15: 500,
         18: 500,
         16: 500})